# Templates

Author(s): Raghav Kansal

In [1]:
from pathlib import Path

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import mplhep as hep
from matplotlib import colors

from boostedhh import utils, hh_vars, plotting
from boostedhh.utils import PAD_VAL, Cutflow
from bbtautau.postprocessing import Samples, Regions, postprocessing
from bbtautau.postprocessing.Samples import CHANNELS, SAMPLES, SIGNALS
import bbtautau.postprocessing.utils as putils
from bbtautau.postprocessing.utils import LoadedSample

import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("boostedhh.utils")
logger.setLevel(logging.DEBUG)

In [2]:
# automatically reloads imported files on edits
%load_ext autoreload
%autoreload 2

In [3]:
MAIN_DIR = Path("../../../")
CHANNEL = CHANNELS["he"]  # options: "hh", "he", "hm"

tag = "25Apr23"
year = "2022"

plot_dir = MAIN_DIR / f"plots/Templates/{tag}/{CHANNEL.key}"
plot_dir.mkdir(parents=True, exist_ok=True)

templates_dir = Path(f"./templates/{tag}/{CHANNEL.key}")
(templates_dir / "cutflows" / year).mkdir(parents=True, exist_ok=True)

base_dir = Path("/ceph/cms/store/user/rkansal/bbtautau/skimmer/")
data_paths = {
    "signal": base_dir / "25Apr17bbpresel_v12_private_signal",
    "data": base_dir / "25Apr17bbpresel_v12_private_signal",
    "bg": base_dir / "25Apr17bbpresel_v12_private_signal",
}

sigs = {s + CHANNEL.key: SAMPLES[s + CHANNEL.key] for s in SIGNALS}
bgs = {bkey: b for bkey, b in SAMPLES.items() if b.get_type() == "bg"}

In [ ]:
from bbtautau.HLTs import HLTs

trigdict = postprocessing.trigger_filter(HLTs.hlts_list_by_dtype(year), year)
trigdict["signal"]

## Load samples

In [ ]:
# dictionary that will contain all information (from all samples)
events_dict = postprocessing.load_samples(
    year,
    CHANNEL,
    data_paths,
    load_data=True,
    load_bgs=True,
    filters_dict=postprocessing.bb_filters(num_fatjets=3, bb_cut=0.8),
    loaded_samples=True,
)
cutflow = utils.Cutflow(samples=events_dict)
cutflow.add_cut(events_dict, "Preselection", "finalWeight")
cutflow.cutflow

Triggers

In [ ]:
postprocessing.apply_triggers(events_dict, year, CHANNEL)
cutflow.add_cut(events_dict, "Triggers", "finalWeight")
cutflow.cutflow

bbtautau assignment

In [29]:
postprocessing.bbtautau_assignment(events_dict, CHANNEL)

Templates

In [ ]:
# CHANNEL = Samples.CHANNELS["hh"]  # reload for new cuts
templates = postprocessing.get_templates(
    events_dict,
    year,
    sigs,
    bgs,
    CHANNEL,
    postprocessing.shape_vars,
    {},
    pass_ylim=150,
    fail_ylim=1e5,
    sig_scale_dict={f"bbtt{CHANNEL.key}": 300, f"vbfbbtt-k2v0{CHANNEL.key}": 40},
    plot_dir=plot_dir,
    template_dir=templates_dir,
    show=True,
)

In [ ]:
postprocessing.save_templates(
    templates, templates_dir / f"{year}_templates.pkl", True, postprocessing.shape_vars
)